### 휴먼지능정보공학과 201910787 김영서

앞서 Multiply7을 구현해 놓았다.

이 컨트랙에 이벤트를 추가해서 수정해보자.

geth 또는 ganache 가운데 어느 것이나 선택해서 실행해도 좋다.

아래 주어진 미완성 코드를 구현해서 노드에서 이벤트를 발생시켜 본다.

* 함수는 param4인자를 받아서 곱하기 7 연산을 한다. 8과 16의 연산을 실행하자. 그 결과는 56과 112가 출력될 것이다.

* 이벤트로 넘겨주는 매개변수 param1은 누가 이 함수를 호출했는지 (address), param2는 언제 호출되었는지 (timestamp), 마지막으로 param3는 연산결과를 출력한다.



* 미완성 코드

contract Multiply7Event {

    event Print(param1, param2, param3);

    function multiply(param4)

}

In [7]:
%%writefile src/Multiply7Event.sol
pragma solidity 0.5.10;
contract Multiply7Event {
    event Print(address param1,uint256 param2,uint param3);
    function multiply(uint param4) public {
        emit Print(msg.sender, now, param4*7);
    }
}

Overwriting src/Multiply7Event.sol


In [8]:
!solc --gas --abi --bin src/Multiply7Event.sol


======= src/Multiply7Event.sol:Multiply7Event =======
Gas estimation:
construction:
   105 + 52400 = 52505
external:
   multiply(uint256):	1849
Binary: 
6080604052348015600f57600080fd5b506101068061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea265627a7a7230582004e1bdf0cb3ec8e7edfa598ef4ef3d99e3368c18e242aafb0530ee8f1cd1f20364736f6c634300050a0032
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"param1","type":"address"

In [9]:
%%writefile src/Multiply7EventDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"param1","type":"address"},{"indexed":false,"name":"param2","type":"uint256"},{"indexed":false,"name":"param3","type":"uint256"}],"name":"Print","type":"event"}];
var _bin = "0x" + "6080604052348015600f57600080fd5b506101068061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b81019080803590602001909291905050506058565b005b7f91da4985ab616136202f4e81fd2d8cac1eb12591132d609cece407f7c6fb9205334260078402604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a15056fea265627a7a72305820ecbeace7a9edb7d109c90b66cb5e9b21a149b35c835fba1b6670021e5234e31764736f6c634300050a0032";
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Multiply7EventDeploy.js


In [10]:
!node src/Multiply7EventDeploy.js

Deploying the contract from 0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466
hash: 0x183149c009243035f504e94ff899a7d6559ece5dd213ebf9a9633ee964ef2b58
---> The contract deployed to: 0x2B78826937BCa64f9E45F23aaf47a564A016c594


In [16]:
%%writefile src/Multiply7EventUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray = [{"constant":false,"inputs":[{"name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":false,"name":"param1","type":"address"},{"indexed":false,"name":"param2","type":"uint256"},{"indexed":false,"name":"param3","type":"uint256"}],"name":"Print","type":"event"}];
var _test = new web3.eth.Contract(_abiArray, '0x2B78826937BCa64f9E45F23aaf47a564A016c594');
var event = _test.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const value8 = await _test.methods.multiply(8)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log("\n-------7 multiply by 8--------");
    console.log(value8.events.Print.returnValues);
    console.log("address:",value8.events.Print.returnValues[0]);
    console.log("timestamp:",value8.events.Print.returnValues[1]);
    console.log("result:",value8.events.Print.returnValues[2]);
    
    console.log("\n-------7 multiply by 16--------");
    const value16 = await _test.methods.multiply(16)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
    console.log(value16.events.Print.returnValues);
    console.log("address:",value16.events.Print.returnValues[0]);
    console.log("timestamp:",value16.events.Print.returnValues[1]);
    console.log("result:",value16.events.Print.returnValues[2]);
}
doIt()

Overwriting src/Multiply7EventUse.js


In [17]:
!node src/Multiply7EventUse.js


-------7 multiply by 8--------
Result {
  '0': '0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466',
  '1': '1621599581',
  '2': '56',
  param1: '0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466',
  param2: '1621599581',
  param3: '56'
}
address: 0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466
timestamp: 1621599581
result: 56

-------7 multiply by 16--------
Result {
  '0': '0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466',
  '1': '1621599581',
  '2': '112',
  param1: '0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466',
  param2: '1621599581',
  param3: '112'
}
address: 0x58B409a07C98AD228bEEF4B6AE9D4C050cB32466
timestamp: 1621599581
result: 112
